### Импорты

In [4]:
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          AutoModelForSeq2SeqLM, 
                          AutoConfig)
import torch
import gc

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from huggingface_hub import login
login("YOUR_TOKEN") # Llama + Mistral

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Общие функции

In [7]:
def load_model_and_tokenizer(model_name,
                             use_auth_token=False):
    config = AutoConfig.from_pretrained(model_name, use_auth_token=use_auth_token)
    
    tokenizer = AutoTokenizer.from_pretrained(model_name,
                                              use_auth_token=use_auth_token)

    if config.architectures and 'T5' in config.architectures[0]:
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                                      use_auth_token=use_auth_token).to(device)
    else:
        model = AutoModelForCausalLM.from_pretrained(model_name,
                                                     use_auth_token=use_auth_token).to(device)
    
    return model, tokenizer

In [8]:
def generate_text(model,
                  tokenizer,
                  prompt,
                  temperature=1.0,
                  max_length=100):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=temperature
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [9]:
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

### Vikhr Llama 3.2 1B

In [ ]:
model_name_vikhr_llama1b = "Vikhrmodels/Vikhr-Llama-3.2-1B-instruct"
model_vikhr_llama1b, tokenizer_vikhr_llama1b = load_model_and_tokenizer(model_name_vikhr_llama1b)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_vikhr_llama1b = generate_text(model_vikhr_llama1b,
                                             tokenizer_vikhr_llama1b,
                                             prompt)

print(f"Generated text for {model_name_vikhr_llama1b}: {generated_text_vikhr_llama1b}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated text for Vikhrmodels/Vikhr-Llama-3.2-1B-instruct: Напиши рецепт вкусного борща, который стоит сделать-home. Напишемrecipe с английскими и русскыми именами ингредиентов.

### Рецепт Борща:

**Ингрейденты:**
- 3mediumsweet onions — 250 грамм
- Овощи: картофель (1 шт.), лук репчатый (2 шт.),
- Зелень (укроп, петруш


In [ ]:
clear_memory()

del model_vikhr_llama1b
del tokenizer_vikhr_llama1b

torch.cuda.synchronize()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

Vikhr Llama 3.2 1B выводит некорректные ответы. Не подходит для использования.

### Vikhr Gemma2B Instruct

In [ ]:
model_name_vikhr_gemma2b = "Vikhrmodels/Vikhr-Gemma-2B-instruct"
model_vikhr_gemma2b, tokenizer_vikhr_gemma2b = load_model_and_tokenizer(model_name_vikhr_gemma2b)

In [ ]:
prompt = "Напиши подробный рецепт вкусного борща"
generated_text_vikhr_gemma2b = generate_text(model_vikhr_gemma2b,
                                             tokenizer_vikhr_gemma2b,
                                             prompt,
                                             temperature=1.0,
                                             max_length=500)

print(f"Generated text for {model_name_vikhr_gemma2b}: {generated_text_vikhr_gemma2b}")

Generated text for Vikhrmodels/Vikhr-Gemma-2B-instruct: Напиши подробный рецепт вкусного борща из курицы с использованием качественных продуктов. Не забудь описать технологический процесс приготовления, время каждого этапа, и рекомендуемые продукты. В ответе только рецепт, без дополнительной информации и обсуждений.

Рецепт должен включать информацию о продуктах, которые лучше всего подходят для бортика. Кроме того, стоит включить информацию, как правильно выбрать и разморозить курицу перед приготовлением. Также важно включит информацию по выбору овощей, способ применения специй и добавление масла, чтобы сделать вкуснее. Подумайте и предложите способ приготовления борщу-пасты (например, с добавления соуса для пиццы или даже овощного бульо).
Answer on russian


In [ ]:
clear_memory()

Vikhr Gemma2B Instruct оборачивет промпт в более подробные инструкции, но не выполняет их

### Vikhr Qwen 2.5 0.5B

In [ ]:
model_name_vikhr_qwen05b = "Vikhrmodels/Vikhr-Qwen-2.5-0.5b-Instruct"
model_vikhr_qwen05b, tokenizer_vikhr_qwen05b = load_model_and_tokenizer(model_name_vikhr_qwen05b)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_vikhr_qwen05b = generate_text(model_vikhr_qwen05b,
                                             tokenizer_vikhr_qwen05b,
                                             prompt,
                                             temperature=0.9,
                                             max_length=500)

print(f"Generated text for {model_name_vikhr_qwen05b}: {generated_text_vikhr_qwen05b}")

Generated text for Vikhrmodels/Vikhr-Qwen-2.5-0.5b-Instruct: Напиши рецепт вкусного борща, используя только ингредиенты, которые можно найти в магазине за $0.15.

Рецепт: Свекла - $4.9, Баранина - 30 руб., Курица свежая - дюжина рублей, Морковь - рубль
Инструкция: Никогда не добавляйте молоко и томатный соус в суп, если есть проблемы с водой или мелкий капуста. Подайте блюдо с зеленью


In [ ]:
clear_memory()

Vikhr Qwen 2.5 может выполнять команды, хорошо понимает русский язык, но склонна делать обертку для промпта

### Vikhr Nemo 12B

In [ ]:
model_name_vikhr_nemo12b = "Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24"
model_vikhr_nemo12b, tokenizer_vikhr_nemo12b = load_model_and_tokenizer(model_name_vikhr_nemo12b)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_vikhr_nemo12b, = generate_text(model_vikhr_nemo12b,
                                             tokenizer_vikhr_nemo12b,
                                             prompt)

print(f"Generated text for {model_name_vikhr_nemo12b}: {generated_text_vikhr_nemo12b}")

In [ ]:
clear_memory()

### RuGPT3 Large Based on GPT 2

In [ ]:
model_name_rugpt3l = "ai-forever/rugpt3large_based_on_gpt2"
model_rugpt3l, tokenizer_rugpt3l = load_model_and_tokenizer(model_name_rugpt3l)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_rugpt3l = generate_text(model_rugpt3l,
                                       tokenizer_rugpt3l,
                                       prompt)

print(f"Generated text for {model_name_rugpt3l}: {generated_text_rugpt3l}")

Generated text for ai-forever/rugpt3large_based_on_gpt2: Напиши рецепт вкусного борща

Состав рецепта: 1,5 л холодной воды, 8—10 картофелин, 200 г свеклы, 1 стакан гречневой крупы, 4 луковицы, 3 моркови, пучок петрушки, перец, соль, 2 лавровых листа, 10–15 горошин черного перца, зелень.
Подготовленные овощи залить холодной водой и варить 20 мин. Добавить мелко нарезанную свеклу, нарезанные крупно лук и


In [ ]:
clear_memory()

RuGPT3 Large Based on GPT 2 хорошо выполняет инструкции, но ответы не слишком умные

### Qwen 2.5 05B Instruct

In [ ]:
model_name_qwen05b = "Qwen/Qwen2.5-0.5B-Instruct"
model_qwen05b, tokenizer_qwen05b = load_model_and_tokenizer(model_name_qwen05b)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_qwen05b = generate_text(model_qwen05b,
                                       tokenizer_qwen05b,
                                       prompt)

print(f"Generated text for {model_name_qwen05b}: {generated_text_qwen05b}")

Generated text for Qwen/Qwen2.5-0.5B-Instruct: Напиши рецепт вкусного борща с медом и овощами. Насыпка: мед или йогурт, черные колбаски. Сколько таких дров - 150г с 47г оливкового масла? Доверяете руке, вы ужинаете в теплом домашнем веке. Плюста на столовую полочку.


In [ ]:
clear_memory()

Qwen 2.5 05B Instruct выдает бессмысленный набор слов

### Qwen 2.5 1.5B

In [ ]:
model_name_qwen15b = "Qwen/Qwen2.5-1.5B"
model_qwen15b, tokenizer_qwen15b = load_model_and_tokenizer(model_name_qwen15b)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_qwen15b = generate_text(model_qwen15b,
                                       tokenizer_qwen15b, prompt)

print(f"Generated text for {model_name_qwen15b}: {generated_text_qwen15b}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated text for Qwen/Qwen2.5-1.5B: Напиши рецепт вкусного борща со специями: | Крекера - сайт о кулинарии
10.04.2018 admin
Вам понадобится: 1 кг говядины, 3 штуки картофель, болгарский перец, брокколи, лук репчатый, розмарин, петрушку, сельдерей, корицу, переопим, черный жасмин, специи (песцовку и соду, куркуму, молотые коричневые ягоды, тмина), масло подсолнечное.
Сначала очистите румяные от кожуры и обработайте лапшу в воде. Аккуратно промойте и просушите салатники на бумажном полотенце. Добавьте майонез, выдержите на медленном огне, пока не закипит, посыпьте термосом, откупорьте крышку.
Лучшие блюда с удовольствием поделится моим автором - Викторией Женштейн.
Пирамида ленцола это как линия обожания, а это просто наслаждение. Вот и божий дар.
Разделить латук на две части. На них обжарить оливковое масло, затем поместить по отдельности, ожидая, когда оно остынет, и покрыть его томатной пастой. Положить их на противень, на среднем огонь, довести до кипения, дать готовиться, покрой б

In [ ]:
clear_memory()

Qwen 2.5 1.5B выдает бессмыленный набор из большого числа слов

### mGPT

In [ ]:
model_name_mgpt = "ai-forever/mGPT"
model_mgpt, tokenizer_mgpt = load_model_and_tokenizer(model_name_mgpt)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_mgpt = generate_text(model_mgpt,
                                    tokenizer_mgpt,
                                    prompt)

print(f"Generated text for {model_name_mgpt}: {generated_text_mgpt}")

Generated text for ai-forever/mGPT: Напиши рецепт вкусного борща по этому решебнику
Все реже в России люди предпочитают заправлять борщ традиционными способами. Но не каждый сможет справиться с приготовлением этого простого блюда в домашних условиях. Рецепты, представленные в данном разделе реферата, помогут вам легко приспособить булочку или борщик из ресторана к имеющимся в


In [ ]:
clear_memory()

mGPT генерирует бессмысленный текст

### Llama 3.2 3B Instruct

In [ ]:
model_name_llama3b = "meta-llama/Llama-3.2-3B-Instruct"
model_llama3b, tokenizer_llama3b = load_model_and_tokenizer(model_name_llama3b,
                                                            use_auth_token=True)

In [12]:
prompt = "Напиши рецепт вкусного борща"
generated_text_llama3b = generate_text(model_llama3b,
                                       tokenizer_llama3b, prompt)

print(f"Generated text for {model_name_llama3b}: {generated_text_llama3b}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated text for meta-llama/Llama-3.2-3B-Instruct: Напиши рецепт вкусного борща с луком и картофелем. Эта еда — классическая простая тушенка на простом ноffective. В нее можно добавить другие ингредиенты на свой собственный выбор, но с классическими вариантами этот борщ вежливый и полезный.

## Варианты борشة

## Легкий и быстрый вариант: «Горький борш»

 – 500


In [ ]:
clear_memory()

### Phi 3.5 mini instruct

In [ ]:
model_name_phi = "microsoft/Phi-3.5-mini-instruct"

tokenizer_phi = AutoTokenizer.from_pretrained(model_name_phi, 
                                              trust_remote_code=True,
                                              device_map="auto")
model_phi = AutoModelForCausalLM.from_pretrained(model_name_phi,
                                                 trust_remote_code=True).to(device)

# model_phi, tokenizer_phi = load_model_and_tokenizer(model_name_phi)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_phi = generate_text(model_phi,
                                   tokenizer_phi,
                                   prompt)

print(f"Generated text for {model_name_phi}: {generated_text_phi}")

In [ ]:
clear_memory()

### AYA 101

In [ ]:
model_name_aya = "CohereForAI/aya-101"
model_aya, tokenizer_aya = load_model_and_tokenizer(model_name_aya)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_aya = generate_text(model_aya,
                                   tokenizer_aya,
                                   prompt)

print(f"Generated text for {model_name_aya}: {generated_text_aya}")

In [ ]:
clear_memory()

### mT0 xxl

In [ ]:
model_name_mt0xxl = "bigscience/mt0-xxl"
model_mt0xxl, tokenizer_mt0xxl = load_model_and_tokenizer(model_name_mt0xxl)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_mt0xxl = generate_text(model_mt0xxl,
                                      tokenizer_mt0xxl,
                                      prompt)

print(f"Generated text for {model_name_mt0xxl}: {generated_text_mt0xxl}")

In [ ]:
clear_memory()

### mT0 large

In [ ]:
model_name_mt0large = "bigscience/mt0-large"
model_mt0large, tokenizer_mt0large = load_model_and_tokenizer(model_name_mt0large)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_mt0large = generate_text(model_mt0large,
                                        tokenizer_mt0large,
                                        prompt)

print(f"Generated text for {model_name_mt0large}: {generated_text_mt0large}")

In [ ]:
clear_memory()

### Bloomz-7B1 mt

In [ ]:
model_name_bloomz7b = "bigscience/bloomz-7b1-mt"
model_bloomz7b, tokenizer_bloomz7b = load_model_and_tokenizer(model_name_bloomz7b)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_bloomz7b = generate_text(model_bloomz7b,
                                        tokenizer_bloomz7b,
                                        prompt,
                                        temperature=0.9,
                                        max_length=500)

print(f"Generated text for {model_name_bloomz7b}: {generated_text_bloomz7b}")

In [ ]:
clear_memory()

### Bloomz 1B7

In [ ]:
model_name_bloomz1b = "bigscience/bloomz-1b7"
model_bloomz1b, tokenizer_bloomz1b = load_model_and_tokenizer(model_name_bloomz1b)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_bloomz1b = generate_text(model_bloomz1b,
                                        tokenizer_bloomz1b,
                                        prompt,
                                        temperature=1.2,
                                        max_length=500)

print(f"Generated text for {model_name_bloomz1b}: {generated_text_bloomz1b}")

Generated text for bigscience/bloomz-1b7: Напиши рецепт вкусного борща (the full form of your order). Put the final quote at the bottom of the document. Add your payment card number as a last (optional).


In [ ]:
clear_memory()

Bloomz 1B7 не использует русский язык для ответа

### Mistral 7B v0.1

In [ ]:
model_name_mistral7b = "mistralai/Mistral-7B-v0.1"
model_mistral7b, tokenizer_mistral7b = load_model_and_tokenizer(model_name_mistral7b,
                                                                use_auth_token=True)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_mistral7b = generate_text(model_mistral7b,
                                         tokenizer_mistral7b, prompt)

print(f"Generated text for {model_name_mistral7b}: {generated_text_mistral7b}")

In [ ]:
clear_memory()

### Mistral Nemo Base

In [ ]:
model_name_mistral_nemo = "mistralai/Mistral-Nemo-Base-2407 "
model_mistral_nemo, tokenizer_mistral_nemo = load_model_and_tokenizer(model_name_mistralnemo,
                                                                    use_auth_token=True)

In [ ]:
prompt = "Напиши рецепт вкусного борща"
generated_text_mistral_nemo = generate_text(model_mistral_nemo,
                                            tokenizer_mistral_nemo, 
                                            prompt)

print(f"Generated text for {model_name_mistral_nemo}: {generated_text_mistral_nemo}")

In [ ]:
clear_memory()